### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input
from keras.optimizers import Adam
import pickle

### Preprocess Data

In [2]:
train = pd.read_csv('Synthetic-Persona-Chat_train.csv')

In [3]:
vald = pd.read_csv('Synthetic-Persona-Chat_valid.csv')

In [4]:
test= pd.read_csv('Synthetic-Persona-Chat_test.csv')

In [5]:
train

,user 1 personas,user 2 personas,Best Generated Conversation
0,I am 32.\nI do not want a job.\nI play video g...,My favorite drink is iced coffee.\nI have a bl...,User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...
1,I am 32.\nI play video games all day.\nI still...,I have a ford f150.\nI like ford cars.\nMy tru...,"User 1: Hey, how's it going?\nUser 2: Good, I'..."
2,I am 32.\nI play video games all day.\nI still...,I can recite the movie young frankenstein word...,"User 1: Hi, my name is John. What's your name?..."
3,I write.\nI work at mcdonald s.\nI watch youtu...,I want to move.\nI don t like feeling controll...,User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...
4,I am bald.\nI like to swim.\nMy favorite drink...,My favorite store is american eagle.\nI enjoy ...,User 1: Hello!\nUser 2: Hi!\nUser 1: What do y...
...,...,...,...
8933,"My parents used to work in politics , until th...",I am in a very intimate and loving relationshi...,"User 1: Hey, what's up?\nUser 2: Not much, jus..."
8934,"My parents used to work in politics , until th...",My mind is set on things above.\nI hate evil.\...,User 1: Hey!\nUser 2: Hello!\nUser 1: What do ...
8935,"I went on welfare last month , which makes me ...",I got married last year.\nI am a hair stylist....,"User 1: Hey, I'm [name].\nUser 2: Hi, I'm [nam..."
8936,"I went on welfare last month , which makes me ...",I wish i made more money.\nI have a strange ob...,"User 1: Hi, I'm [user 1].\n\nUser 2: Hi, I'm [..."


In [6]:
vald

,user 1 personas,user 2 personas,Best Generated Conversation
0,I love to bake cookies.\nI have a dogs.\nThe c...,I am a boy.\nI can move objects with my mind.\...,User 1: Hi!\nUser 2: Hello!\nUser 1: What is y...
1,I am an animal activist.\nThe holidays make me...,I feel old.\nI am currently in a juvenile dete...,"User 1: Hi, how are you?\nUser 2: I'm doing ok..."
2,Cheeseburgers are my favorite food.\nI like wa...,I feel old.\nI am currently in a juvenile dete...,"User 1: Hello, my name is (name).\nUser 2: Hel..."
3,I plan to go to business school next year.\nMy...,I am a male.\nI own a house in florida.\nI hav...,User 1: I'm looking forward to going to busine...
4,I can drive a tractor.\nMy favorite color is r...,I am a male.\nI have a children and a dogs.\nI...,User 1: I'm so glad we're both going to the co...
...,...,...,...
995,I attend church every week.\nI work as a schoo...,I live in california but the recording artist ...,"User 1: Hi, I'm a school teacher and I attend ..."
996,I am in an open polyamorous relationship.\nI a...,I live in california but the recording artist ...,"User 1: Hi, I'm [user 1 name].\nUser 2: Hi, I'..."
997,I do not pick up my toys.\nI am four.\nI have ...,I live in california but the recording artist ...,User 1: Hi!\nUser 2: Hi! How are you doing tod...
998,I am an omnivore.\nI just bought my first home...,I have an internet addiction and spend a lot o...,User 1: Hello! What are you interested in?\nUs...


In [7]:
test

,user 1 personas,user 2 personas,Best Generated Conversation
0,I just bought a brand new house.\nI like to da...,I love to meet new people.\nI have a turtle na...,"User 1: Hi, I'm [User 1's name]. What's your n..."
1,I am an old man.\nI have a wheelchair that is ...,I drive a ford pickup truck.\nI am very conser...,User 1: Hi there!\nUser 2: Hello.\nUser 1: I'm...
2,I met my best friend in kindergarten.\nI am of...,I have two dogs and one cat.\nI work as a nurs...,User 1: Hi! What do you do for work?\nUser 2: ...
3,I love having facial hair.\nI hope to retire s...,I am happy being single and alone.\nI only dri...,User 1: How are you doing today?\nUser 2: I'm ...
4,I love watching movies and tv.\nI have a husba...,I am afraid of heights.\nI love animals and ha...,User 1: Hi there!\nUser 2: Hi there!\nUser 1: ...
...,...,...,...
963,I work out every day.\nMy favorite musician is...,I got a job working in advertising last year.\...,"User 1: Hi, my name is [user 1's name].\nUser ..."
964,I lift weights every weekend.\nI am big and ta...,I am currently attending school.\nI like music...,"User 1: Hey, I'm [user 1].\n\nUser 2: Hi, I'm ..."
965,I am considering going to college.\nIts a dead...,I do not eat anything sweet.\nI do not drive b...,"User 1: Hi, my name is [user 1's name].\nUser ..."
966,I have an exotic fish aquarium.\nI collect vin...,I only drink water.\nI do not want children.\n...,User 1: Hey there!\nUser 2: Hello!\nUser 1: I'...


In [8]:
test.dropna(inplace=True)

_______________________________________________________________________

### Cleaning and collecting conversations 

##### Train

In [9]:
count=0
for row in train.iterrows():
    if count==1:
        break
    conv=row[1][2]
    conv=conv.split("\n")
    for i in range(0,len(conv),2):
        print("<SOS> "+conv[i][8:]+" <EOS>")
        if i+1<len(conv):
            print("<SOS> "+conv[i+1][8:]+" <EOS>")
    if len(conv)%2!=0:
        print(len(conv))
    count+=1
    

<SOS> Hi! I'm [user 1's name]. <EOS>
<SOS> Hi [user 1's name], I'm [user 2's name]. <EOS>
<SOS> What do you do for fun? <EOS>
<SOS> I like to play video games, go to the beach, and read. <EOS>
<SOS> I like to play video games too! I'm not much of a reader, though. <EOS>
<SOS> What video games do you like to play? <EOS>
<SOS> I like to play a lot of different games, but I'm really into competitive online games right now. <EOS>
<SOS> I'm not really into competitive games, I like to play more relaxing games. <EOS>
<SOS> That's cool. What kind of relaxing games do you like to play? <EOS>
<SOS> I like to play puzzle games, simulation games, and story-based games. <EOS>
<SOS> I've never been much of a puzzle game person, but I do like simulation games and story-based games. <EOS>
<SOS> Nice! What's your favorite simulation game? <EOS>
<SOS> I like Stardew Valley a lot. It's a farming game, but it's also really relaxing and fun. <EOS>
<SOS> I've heard good things about that game. I might have

C:\Users\adnan\AppData\Local\Temp\ipykernel_2072\3633430242.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv=row[1][2]


In [10]:
person1_train=[]
person2_train=[]
for row in train.iterrows():
    conv=row[1][2]
    conv=conv.split("\n")
    if len(conv)%2 == 0:
        for i in range(0,len(conv),2):
            person1_train.append("<SOS> "+conv[i][8:]+" <EOS>")
            if i+1<len(conv):
                person2_train.append("<SOS> "+conv[i+1][8:]+" <EOS>")

C:\Users\adnan\AppData\Local\Temp\ipykernel_2072\2890846682.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv=row[1][2]


In [11]:
len(person1_train),len(person2_train)

(66455, 66455)

______________________________________________________________________

##### Validation

In [12]:
person1_vald=[]
person2_vald=[]
for row in vald.iterrows():
    conv=row[1][2]
    conv=conv.split("\n")
    if len(conv)%2 == 0:
        for i in range(0,len(conv),2):
            person1_vald.append("<SOS> "+conv[i][8:]+" <EOS>")
            if i+1<len(conv):
                person2_vald.append("<SOS> "+conv[i+1][8:]+" <EOS>")

C:\Users\adnan\AppData\Local\Temp\ipykernel_2072\1743355849.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conv=row[1][2]


In [13]:
len(person1_vald),len(person2_vald)

(7824, 7824)

___________________________________________________________________

### Tokenizing the data

##### Train

In [16]:
tokenizer = Tokenizer()

In [17]:
all_texts = person1_train + person2_train

tokenizer.fit_on_texts(all_texts)

In [18]:
with open('tokenizer_1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
person1_sequences = tokenizer.texts_to_sequences(person1_train)
person2_sequences = tokenizer.texts_to_sequences(person2_train)

In [20]:
# Example of the first few tokenized sequences
print(person1_sequences[:2])
print(person2_sequences[:2])

[[2, 49, 1], [2, 35, 120, 1]]
[[2, 190, 1], [2, 54, 105, 86, 495, 57, 1]]


In [21]:
word_index = tokenizer.word_index
print("Vocabulary size:", len(word_index))

Vocabulary size: 9982


In [22]:
word_index

{'eos': 1,
 'sos': 2,
 'i': 3,
 'to': 4,
 'a': 5,
 'you': 6,
 "i'm": 7,
 'like': 8,
 'do': 9,
 'the': 10,
 'and': 11,
 'of': 12,
 'too': 13,
 'love': 14,
 'it': 15,
 'what': 16,
 'so': 17,
 "it's": 18,
 "that's": 19,
 'for': 20,
 'that': 21,
 'my': 22,
 'fun': 23,
 'great': 24,
 'favorite': 25,
 'me': 26,
 'your': 27,
 'is': 28,
 'have': 29,
 "i've": 30,
 'but': 31,
 'in': 32,
 'are': 33,
 'oh': 34,
 "what's": 35,
 'lot': 36,
 'good': 37,
 'about': 38,
 'really': 39,
 'go': 40,
 'sounds': 41,
 'cool': 42,
 'sure': 43,
 'name': 44,
 'with': 45,
 'be': 46,
 'read': 47,
 'play': 48,
 'hi': 49,
 'thanks': 50,
 'always': 51,
 'should': 52,
 'nice': 53,
 'not': 54,
 'also': 55,
 'can': 56,
 'out': 57,
 'work': 58,
 "they're": 59,
 'games': 60,
 'been': 61,
 'all': 62,
 'time': 63,
 'we': 64,
 "i'll": 65,
 'know': 66,
 'think': 67,
 'user': 68,
 'yeah': 69,
 'glad': 70,
 'movies': 71,
 'was': 72,
 'music': 73,
 'on': 74,
 'things': 75,
 'them': 76,
 'at': 77,
 'kind': 78,
 'one': 79,
 'never'

##### shifting

In [23]:
encoder_input_data=person1_sequences
decoder_data=person2_sequences

In [24]:
decoder_input_data = [sent[:-1] for sent in decoder_data]

decoder_target_data = [sent[1:] for sent in decoder_data]

##### padding

In [25]:
max_seq_length = max(max(len(seq) for seq in person1_sequences),
                     max(len(seq) for seq in person2_sequences))
max_seq_length

137

In [26]:
encoder_input_data_padded = pad_sequences(encoder_input_data, maxlen=max_seq_length, padding='post')
decoder_input_data_padded= pad_sequences(decoder_input_data, maxlen=max_seq_length, padding='post')
decoder_target_data_padedd = pad_sequences(decoder_target_data, maxlen=max_seq_length, padding='post')

In [27]:
# Reshape decoder_target_data to make it suitable for sparse_categorical_crossentropy loss
decoder_target_data_padedd = np.expand_dims(decoder_target_data_padedd, -1)

In [28]:
# Check the shapes again
print(f"encoder_input_data.shape: {encoder_input_data_padded.shape}")
print(f"decoder_input_data.shape: {decoder_input_data_padded.shape}")
print(f"decoder_target_data.shape: {decoder_target_data_padedd.shape}")

encoder_input_data.shape: (66455, 137)
decoder_input_data.shape: (66455, 137)
decoder_target_data.shape: (66455, 137, 1)


________________________________________________________________

##### Validation

In [29]:
validation_person1_sequences = tokenizer.texts_to_sequences(person1_vald)
validation_person2_sequences = tokenizer.texts_to_sequences(person2_vald)

In [30]:
encoder_input_data_valda=validation_person1_sequences
decoder_data_valda=validation_person2_sequences

In [31]:
decoder_input_data_valda=[sent[:-1]for sent in decoder_data_valda]
decoder_target_data_valda=[sent[1:]for sent in decoder_data_valda]

In [32]:
max_seq_length_validation = max(max(len(seq) for seq in validation_person1_sequences),
                     max(len(seq) for seq in validation_person2_sequences))

In [33]:
encoder_input_data_val_padded = pad_sequences(encoder_input_data_valda, maxlen=max_seq_length_validation, padding='post')
decoder_input_data_val_padded = pad_sequences(decoder_input_data_valda, maxlen=max_seq_length_validation, padding='post')
decoder_target_data_val_padded = pad_sequences(decoder_target_data_valda, maxlen=max_seq_length_validation, padding='post')

In [34]:
# Reshape decoder_target_data to make it suitable for sparse_categorical_crossentropy loss
decoder_target_data_val_padded = np.expand_dims(decoder_target_data_val_padded, -1)

In [35]:
# Check the shapes again
print(f"encoder_input_data_validation.shape: {encoder_input_data_val_padded.shape}")
print(f"decoder_input_data_validation.shape: {decoder_input_data_val_padded.shape}")
print(f"decoder_target_data_validation.shape: {decoder_target_data_val_padded.shape}")

encoder_input_data_validation.shape: (7824, 61)
decoder_input_data_validation.shape: (7824, 61)
decoder_target_data_validation.shape: (7824, 61, 1)


### Modeling

In [36]:
# Parameters
embedding_dim = 1024
latent_dim = 512
vocab_size = len(word_index) +1
#dropout_rate = 0.1 # dropout mae accure a problem
bat_size = 256
learning_rate=0.001

In [37]:
# Define the encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim,mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim ,return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim,mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim,return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer= optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])

# Model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 1024)   10222592    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 1024)   10222592    ['input_2[0][0]']                
                                                                                              

In [38]:
model.fit([encoder_input_data_padded,decoder_input_data_padded], decoder_target_data_padedd,
          batch_size=bat_size,
          epochs=25,
          validation_data=([encoder_input_data_val_padded, decoder_input_data_val_padded], decoder_target_data_val_padded))

Epoch 1/25
260/260 [==============================] - 51s 170ms/step - loss: 0.3747 - acc: 0.1753 - val_loss: 0.6426 - val_acc: 0.2829
Epoch 2/25
260/260 [==============================] - 42s 162ms/step - loss: 0.2528 - acc: 0.3543 - val_loss: 0.5060 - val_acc: 0.3922
Epoch 3/25
260/260 [==============================] - 42s 161ms/step - loss: 0.2098 - acc: 0.4325 - val_loss: 0.4499 - val_acc: 0.4516
Epoch 4/25
260/260 [==============================] - 42s 161ms/step - loss: 0.1859 - acc: 0.4797 - val_loss: 0.4186 - val_acc: 0.4776
Epoch 5/25
260/260 [==============================] - 42s 161ms/step - loss: 0.1698 - acc: 0.5080 - val_loss: 0.3984 - val_acc: 0.4960
Epoch 6/25
260/260 [==============================] - 42s 161ms/step - loss: 0.1576 - acc: 0.5303 - val_loss: 0.3845 - val_acc: 0.5106
Epoch 7/25
260/260 [==============================] - 42s 161ms/step - loss: 0.1477 - acc: 0.5501 - val_loss: 0.3753 - val_acc: 0.5206
Epoch 8/25
260/260 [==============================] - 4

In [39]:
model.save('model_embedding_1024.h5')